
Дан датасет для классификации.

Определим один из 7 преобладающих видов деревьев на лесных участках размером 30*30 метров (7 классов).

В качества признаков даны высота над уровнем моря, расстояния до подземных источников и мест лесных пожаров, один из 4 лесных зон, 40 типов почвы (последние 40 колонок) и др.

Описание данных можно найти на https://scikit-learn.org/stable/datasets/real_world.html#forest-covertypes

In [ ]:
# загружаем нужные библиотеки для
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_covtype

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

from collections import Counter

In [2]:
# загрузим датасет из библиотеки датасетов
data = fetch_covtype()

# определяем целевой признак target
features = data['data']
target = data['target']

In [ ]:
print(features.shape)

(581012, 54)


Разделим выборку на тренировочную и тестовую

In [ ]:
Counter(target)
# Как видите, количество представителей классов неравномерное,
# так что попробуем использовать аргумент stratify при разбиении.

Counter({1: 211840, 2: 283301, 3: 35754, 4: 2747, 5: 9493, 6: 17367, 7: 20510})

In [ ]:
# определим количество сплитов
splits = 7

# определим количество фолдов
kfold = StratifiedKFold(n_splits=splits, shuffle=True, random_state=42)
shufflesplit = StratifiedShuffleSplit(n_splits=splits, random_state=42, test_size=7)

print("KFold")
for train_index, test_index in kfold.split(features, target):
    print("TRAIN:", train_index, "TEST:", test_index)

print("Shuffle Split")
for train_index, test_index in shufflesplit.split(features, target):
    print("TRAIN:", train_index, "TEST:", test_index)

KFold
TRAIN: [     1      2      3 ... 581009 581010 581011] TEST: [     0     10     11 ... 580990 581003 581004]
TRAIN: [     0      1      2 ... 581009 581010 581011] TEST: [     7     12     14 ... 580996 581007 581008]
TRAIN: [     0      1      2 ... 581009 581010 581011] TEST: [     9     17     18 ... 580995 580997 580998]
TRAIN: [     0      2      3 ... 581008 581009 581010] TEST: [     1      4     21 ... 580999 581000 581011]
TRAIN: [     0      1      4 ... 581009 581010 581011] TEST: [     2      3      6 ... 581001 581002 581005]
TRAIN: [     0      1      2 ... 581007 581008 581011] TEST: [     8     19     39 ... 581006 581009 581010]
TRAIN: [     0      1      2 ... 581009 581010 581011] TEST: [     5     13     16 ... 580968 580991 580994]
Shuffle Split
TRAIN: [ 63830 350385 512948 ... 276464 287143  52912] TEST: [420449 199566 403915 124205 429795 578448 443779]
TRAIN: [473940 320723  34212 ... 421621 261345 486654] TEST: [275338 503575 562962 183720 368319 315024  

In [ ]:
# разобьем наш датасет на тренировочную и тестовую выборки с учетом стратификации
X_train, X_test, y_train, y_test = train_test_split(features, target,
                                                    train_size=0.67,
                                                    random_state=1,
                                                    stratify=target)

In [ ]:
# выведем нашу тестовую выборку по X
X_test

array([[2.963e+03, 1.300e+01, 8.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.319e+03, 2.650e+02, 1.300e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [3.383e+03, 2.460e+02, 1.700e+01, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       ...,
       [2.998e+03, 2.900e+01, 1.100e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.857e+03, 1.000e+02, 5.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.732e+03, 2.730e+02, 2.200e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00]])

## Построение моделей

Построим две модели: KNN и решающее дерево. Посчитаем ответы на тестовой выборке.

In [ ]:
# используем метод KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
# используем predict для KNN
y_pred1 = knn.predict(X_test)
y_pred1

array([2, 3, 1, ..., 2, 2, 2], dtype=int32)

In [ ]:
print(knn.predict_proba(X_test))

[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [ ]:
# выведем метрики для модели KNN
knn.score(X_test, y_test)

0.96427342046794

In [ ]:
# используем модель решающих деревьев
tree = DecisionTreeClassifier(criterion = "entropy", splitter = "random", max_depth = 2,  min_samples_split = 5,
                              min_samples_leaf = 2, max_features = 2)
tree.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=2, max_features=2, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='random')

In [ ]:
# используем predict для DecisionTreeClassifier
y_pred2 = tree.predict(X_test)
y_pred2

array([2, 2, 2, ..., 2, 2, 2], dtype=int32)

In [ ]:
# посмотрим узлы для DecisionTreeClassifier
tree.tree_.__getstate__()['nodes']

array([( 1,  4, 35,  0.13255438, 1.73866976, 389278, 389278.),
       ( 2,  3,  8, 81.76915402, 1.75853678, 367015, 367015.),
       (-1, -1, -2, -2.        , 2.28234356,  22418,  22418.),
       (-1, -1, -2, -2.        , 1.71063769, 344597, 344597.),
       ( 5,  6, 12,  0.18593276, 0.80757577,  22263,  22263.),
       (-1, -1, -2, -2.        , 0.80857897,  16657,  16657.),
       (-1, -1, -2, -2.        , 0.7341791 ,   5606,   5606.)],
      dtype=[('left_child', '<i8'), ('right_child', '<i8'), ('feature', '<i8'), ('threshold', '<f8'), ('impurity', '<f8'), ('n_node_samples', '<i8'), ('weighted_n_node_samples', '<f8')])

## Подсчет метрик

Посмотрим метрику F1 для полученных предсказаний с помощью функции f1_score из библиотеки sklearn

In [ ]:
# выведем взвешенную f1 метрику по KNN
print(f1_score(y_test, y_pred1, average="weighted"))

0.9641922652706003


In [ ]:
# выведем макро precision_score по KNN
print(precision_score(y_test, y_pred1, average="macro"))

0.9464118279415855


In [ ]:
# выведем макро recall_score по KNN
print(recall_score(y_test, y_pred1, average="macro"))

0.9192432910972546


In [ ]:
# выведем взвешенную f1 метрику по DecisionTreeClassifier
print(f1_score(y_test, y_pred2, average="macro"))

0.12513596437691407


In [ ]:
# выведем макро precision_score по DecisionTreeClassifier
print(precision_score(y_test, y_pred2, average="macro"))

0.1823791401671582


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# выведем макро recall_score по DecisionTreeClassifier
print(recall_score(y_test, y_pred2, average="macro"))

0.15660719251776345


54 - это довольно большая размерность для пространства признаков.

Попробуем уменьшить его любым доступным способом (в том числе, отбросить что-либо, посмотрев на атрибут DecisionTreeClassifier **feature_importances_**)

In [ ]:
# выведем feature_importances
print (tree.feature_importances_)

[0.         0.         0.         0.         0.         0.
 0.         0.         0.2561485  0.         0.         0.
 0.02122846 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.72262303
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.        ]


In [ ]:
# определим модель классификации признаков
model = ExtraTreesClassifier()
model.fit(features, target)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [ ]:
# выведем данные по влиянию признаков
print(model.feature_importances_)

[1.99233955e-01 5.18852529e-02 3.71011820e-02 6.40936205e-02
 5.90170969e-02 1.12054665e-01 4.33861625e-02 4.53654630e-02
 4.25967753e-02 1.05557370e-01 1.08026412e-02 5.96740715e-03
 1.25633474e-02 3.62641495e-02 1.10852792e-03 1.02861703e-02
 2.65300682e-03 1.31725043e-02 7.71244405e-04 3.11765934e-03
 5.38841929e-05 5.25707238e-05 2.37064109e-04 1.25220447e-02
 3.67204566e-03 1.56773820e-02 5.15114240e-03 5.89204374e-04
 4.99885495e-06 9.58884028e-04 1.96634157e-03 6.21991278e-04
 1.19618931e-03 2.25210337e-03 1.16008522e-03 1.88366877e-02
 1.23900246e-02 4.41183738e-03 2.65592661e-04 8.67699961e-04
 8.45957331e-04 3.45983188e-04 4.96042364e-03 3.54925903e-03
 3.32125557e-03 4.11684424e-03 3.88271252e-03 8.92683389e-04
 2.15287938e-03 1.20821246e-04 7.82158104e-04 1.35941377e-02
 1.36971980e-02 7.85171131e-03]


## Выводы

На основании метрик, полученных в проведенном анализе, качество результатов работы моделей для данной задачи классификации показали, что KNN предпочтительнее чем DecisionTree, т.к. метрики по модели выше
